# マルコフ連鎖モンテカルロ法

ベクトルや行列を太文字表記していないので要注意

In [1]:
# 必要なライブラリをインポート
from tqdm import trange
import numpy as np
import scipy.stats as st
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

## マルコフ連鎖と不変分布

### マルコフ連鎖の概要
確率変数の系列$\{X_t\}_{t=0}^{\infty}$に対して，$\{X_s\}_{s=0}^{t-1}$の実現値$\{x_s\}_{s=0}^{t-1}$が与えられた下で$X_t$が部分集合$A \subseteq \chi$内に値をとる条件付き確率が以下のように表現されるとき，$\{X_t\}_{t=0}^{\infty}$は**マルコフ連鎖**であるという．
$$
    Pr\{ X_t \in A| X_0=x_0, \dotsc , X_{t-1}=x_{t=1} \}
    = Pr\{ X_t \in A| X_{t-1}=x_{t-1} \} \tag{1-1}
$$
> 要は，確率変数$X_t$の条件付き確率分布は直近の確率変数の実現値$x_{t-1}$にのみ依存している．

マルコフ連鎖における重要な概念
- 斉時性: (1-1)式の条件付き確率が時点$t$に依存しないこと
- 規約性: 確率分布$f$で実現しうる$\chi$内のいかなる場所にもマルコフ連鎖が到達可能であること
- 周期性: 一定の時間を経過しないとマルコフ連鎖が再び訪れることができない場所が$\chi$内に存在すること
- 再帰性: 確率分布$f$で実現しうる$\chi$内のいかなる場所にもマルコフ連鎖が有限時間内に必ず訪れること

$X_t$の$\{x_s\}_{s=0}^{t-1}$が与えられた下での条件付き確率分布は以下の通り．
$$
    f_t(x_t|x_0, \dotsc, x_{t-1}) = f(x_t|t_{t-1}) \tag{1-2}
$$
マルコフ連鎖が斉時的であるとき，(1-2)式の右辺の関数形は時点$t$に依存せず，$\{X_s\}_{s=0}^t$のどう次確率分布は以下のように与えられる．
$$
    f(x_0, \dotsc, x_t)
    = f_0(x_0) \prod_{s=1}^t f(x_s|x_{s-1}) \tag{1-3}
$$
この$f(x_t|x_{t-1})$をマルコフ連鎖の**遷移核**と呼ぶ．

### マルコフ連鎖の不変分布
以下を満たす$\bar{f}$をマルコフ連鎖の**不変分布**または**定常分布**という．
$$
    \bar{f}(\tilde{x})
    = \int_{\chi} \bar{f}(x)K(x, \tilde{x})dx, 
    あるいは \
    \bar{f} = \bar{f} \circ K, \tag{1-4}
$$
ここで$K$はマルコフ連鎖の遷移核を表す．


### マルコフ連鎖からの乱数系列の生成
斉時的なマルコフ連鎖は$f_0$と$K$にのみ依存するという簡単な構造をしており，$f_0$と$K$からの乱数生成が可能であれば，マルコフ連鎖からの乱数生成を以下の手順で行うことができる．
1. $t=1$として$\tilde{x}_0 \gets f_0(x_0)$．
2. $\tilde{x}_t \gets K(\tilde{x}_{t-1}, x_t)$．
3. $t$を$1$増やして手順2に戻る

マルコフ連鎖が$\bar{f}$に収束したと判断されるまで乱数$\{\tilde{x_t}\}_{t=0}^{t^*}$を生成し続けることを**検査稼働期間**（バーンイン）と呼ぶ．そして，$\bar{f}$に収束したと判断した回以降のマルコフ連鎖から生成した$\{\tilde{x}_t\}_{t={t^*}+1}^{{t^*}+T}$は$\bar{f}$からのモンテカルロ標本として利用可能である．

不変分布への収束後のサンプリングは大きく分けて(i)多重連鎖法，(ii)単一連鎖法 の2通り

(i)多重連鎖法の手順
1. $\tilde{x}_0 \gets f_0(x_0)$．
2. マルコフ連鎖から分布が収束するまで乱数を生成する．
3. 最後の$\tilde{x}_t$を保存し，手順1に戻る．

(ii)単一連鎖法の手順
1. $\bar{f}$の取りうる領域から$\tilde{x}_0$を選ぶ．
2. マルコフ連鎖から分布が収束するまで乱数を生成する．
3. 必要な数の$\tilde{x}_t$を続けてマルコフ連鎖から生成する．

## メトロポリス-ヘイスティングズ・アルゴリズム

## ギブズ・サンプラー